## Machine Learning Project on UCI Parkinsons Telemonitoring Data Set

Importación de librerías

In [ ]:
#!pip install qgrid

In [1]:
from __future__ import division

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.matlib as matlib

from numpy import random
from numpy import round
import math

import pandas as pd
import qgrid

import scipy as sc
from scipy.spatial import distance
from scipy import stats

#algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

Cargamos la base de datos

In [2]:
#Cargamos la bd que está en un archivo .data y ahora la podemos manejar de forma matricial
db = np.loadtxt('DB/parkinsons_updrs.data', delimiter=',', skiprows=1)  # Assuming ',' delimiter

#X: Toma todas las filas (muestras) y las columnas 6-21 (características)
X = db[:,6:22]
#Y: Toma todas las filas y la columna 4, corresponde a la salida de la regresión
Y = db[:,4]
#G: Toma todas las filas y la columna 0, corresponde a la asociación en grupos del dataset
G = db[:,0]

In [3]:
print(X.shape)
print(Y.shape)
print(G.shape)

(5875, 16)
(5875,)
(5875,)


Medidas de error

In [4]:
#Mean Absolute Percentage Error
def MAPE(Y, Y_est):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape

#### Regresión Lineal Múltiple

In [6]:
# MAE, R2, RMSE

from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score, mean_squared_error

# Normalizamos los datos
scaler = preprocessing.StandardScaler().fit(X)
X_norm = scaler.transform(X)

random.seed(19680801)

iterations = 10
gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
index = gss.split(X, Y, G)

# modelo
model = Pipeline([('poly', PolynomialFeatures()), ('linear', LinearRegression(fit_intercept=False))])
#print(model.get_params().keys())

parameters = {'poly__degree': [1,2,3,4]}

# métricas de error
# greater_is_better=True by default
mae = make_scorer(mean_absolute_error)
r2 = make_scorer(r2_score)
rmse = make_scorer(mean_squared_error, squared=False)

scores = {'mae':mae,'r2':r2,'rmse':rmse}

# Run the grid search
grid_obj = GridSearchCV(model, parameters, scoring=scores, cv=index, return_train_score=True, refit=False)
grid_obj = grid_obj.fit(X_norm, Y)

In [7]:
outcomes = pd.DataFrame(grid_obj.cv_results_)
outcomes = outcomes[['params', 'mean_train_mae', 'mean_train_r2', 'mean_train_rmse',
            'mean_test_mae', 'std_test_mae', 'mean_test_r2', 'std_test_r2', 'mean_test_rmse', 'std_test_rmse']]
outcomes

,params,mean_train_mae,mean_train_r2,mean_train_rmse,mean_test_mae,std_test_mae,mean_test_r2,std_test_r2,mean_test_rmse,std_test_rmse
0,{'poly__degree': 1},6.351467,0.137742,7.524888,7.113906,0.258448,-1.907575e-01,1.597205e-01,8.577520e+00,4.485203e-01
1,{'poly__degree': 2},5.461242,0.316153,6.700819,8.240104,1.386231,-2.519604e+00,4.355412e+00,1.299009e+01,7.007837e+00
2,{'poly__degree': 3},4.150653,0.560697,5.368819,42.478125,51.503028,-1.675949e+04,3.889956e+04,5.516074e+02,8.421256e+02
3,{'poly__degree': 4},0.000259,1.000000,0.000496,96705.654730,112886.283714,-7.763870e+10,8.642567e+10,1.706706e+06,1.352049e+06


#### Ventana de Parzen

In [14]:
def kernel_gaussiano(x):
    return (np.exp((-0.5)*x**2))

def Nadaraya_Watson(X_train, Y_train, X_val, h): # h equivale al ancho de la ventana o kernel
    Nval = len(X_val)
    Ntrain = len(X_train)
    Y_val = np.zeros(Nval)
    
    for i in range(Nval):
        muestra = X_val[i,:]
        numerador = 0
        denominador = 0
        for j in range(Ntrain):
            distancia = distance.euclidean(muestra, X_train[j])/h
            peso = kernel_gaussiano(distancia)
            numerador += peso * Y_train[j]
            denominador += peso
        Y_val[i] = numerador/denominador
    #Se retorna un vector que contiene las predicciones para cada una de las muestras en X_val, en el mismo orden.      
    return Y_val

In [15]:
# MAE, MAPE

from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error

def executePARZEN_NW(h):
    
    iterations = 10
    errorMAE = np.zeros(iterations)
    errorMAPE = np.zeros(iterations)
    
    random.seed(19680801)
    gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
    j=0
    for train_idx, test_idx in gss.split(X, Y, G):
        Xtrain = X[train_idx,:]
        Ytrain = Y[train_idx]
        Xtest = X[test_idx,:]
        Ytest = Y[test_idx]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        Xtrain = sc.stats.stats.zscore(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        Yest = Nadaraya_Watson(Xtrain, Ytrain, Xtest, h)

        #Evaluamos las predicciones del modelo con los datos de test
        errorMAE[j] = mean_absolute_error(Ytest, Yest)
        errorMAPE[j] = MAPE(Ytest, Yest)
        j += 1

    mae = np.mean(errorMAE)
    std_mae = np.std(errorMAE)
    mape = np.mean(errorMAPE)
    std_mape = np.std(errorMAPE)
    
    return(round(mae,4), round(std_mae,4), round(mape,4), round(std_mape,4))


In [16]:
df_types = pd.DataFrame({
    'Ancho del kernel' : pd.Series(['0.5', '1', '2', '5','10'])})
df_types["MAE"] = ""
df_types["std_MAE"] = ""
df_types["MAPE"] = ""
df_types["std_MAPE"] = ""
df_types.set_index(['Ancho del kernel'], inplace=True)

valoresH = np.array([0.5,1,2,5,10])
for i in range(np.size(valoresH)):
    mae, std_mae, mape, std_mape  = executePARZEN_NW(valoresH[i])
    df_types["MAE"][i] = mae
    df_types["std_MAE"][i] = std_mae
    df_types["MAPE"][i] = mape
    df_types["std_MAPE"][i] = std_mape
    
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

C:\Users\Kevin\Anaconda3\lib\site-packages\qgrid\grid.py:827: DeprecationWarning: QgridWidget._df_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _df_changed(self):


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [17]:
qgrid_widget.get_changed_df()

,MAE,std_MAE,MAPE,std_MAPE
Ancho del kernel,,,,
0.5,10.1831,1.5811,0.4315,0.0465
1,7.4617,0.9121,0.3913,0.0556
2,7.0454,0.6787,0.4158,0.0846
5,6.9934,0.6175,0.4256,0.0902
10,6.9817,0.5982,0.4278,0.0914


#### Redes Neuronales Artificiales

In [ ]:
# MAE, MAPE, RMSE

from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error

# Normalizamos los datos
scaler = preprocessing.StandardScaler().fit(X)
X_norm = scaler.transform(X)

random.seed(19680801)

iterations = 10
gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
index = gss.split(X, Y, G)

model = MLPRegressor(activation='tanh')

parameters = {'hidden_layer_sizes': [(8),(16),(32),(8,8),(16,16),(32,32),(8,8,8),(16,16,16),(32,32,32)],
             'max_iter': [500,1000,1500]}

# métricas de error
# greater_is_better=True by default
mae = make_scorer(mean_absolute_error)
mape = make_scorer(MAPE)
rmse = make_scorer(mean_squared_error, squared=False)

scores = {'mae':mae,'mape':mape,'rmse':rmse}

# Run the grid search
grid_obj = GridSearchCV(model, parameters, scoring=scores, cv=index, return_train_score=True, refit=False, verbose=12)
grid_obj = grid_obj.fit(X_norm, Y)

In [6]:
outcomes = pd.DataFrame(grid_obj.cv_results_)
outcomes = outcomes[['params', 'mean_train_mae', 'mean_train_mape', 'mean_train_rmse',
            'mean_test_mae', 'std_test_mae', 'mean_test_mape', 'std_test_mape', 'mean_test_rmse', 'std_test_rmse']]
outcomes

,params,mean_train_mae,mean_train_mape,mean_train_rmse,mean_test_mae,std_test_mae,mean_test_mape,std_test_mape,mean_test_rmse,std_test_rmse
0,"{'hidden_layer_sizes': 8, 'max_iter': 500}",5.562746,0.342647,6.875116,7.547490,0.465845,0.445809,0.095121,9.136517,0.529503
1,"{'hidden_layer_sizes': 8, 'max_iter': 1000}",5.406609,0.333261,6.705013,7.631354,0.584386,0.449369,0.097706,9.233823,0.663682
2,"{'hidden_layer_sizes': 8, 'max_iter': 1500}",5.374483,0.331953,6.685240,7.670587,0.564504,0.451995,0.097002,9.247882,0.600639
3,"{'hidden_layer_sizes': 16, 'max_iter': 500}",5.382395,0.332646,6.684793,7.587403,0.556264,0.447069,0.098077,9.178715,0.594459
4,"{'hidden_layer_sizes': 16, 'max_iter': 1000}",5.141643,0.318138,6.447493,7.737584,0.532623,0.455237,0.104038,9.353877,0.584137
5,"{'hidden_layer_sizes': 16, 'max_iter': 1500}",5.022856,0.311153,6.331027,7.852918,0.522782,0.459786,0.106736,9.509979,0.574289
6,"{'hidden_layer_sizes': 32, 'max_iter': 500}",5.136924,0.318805,6.423922,7.637718,0.519401,0.450205,0.099643,9.230038,0.557211
7,"{'hidden_layer_sizes': 32, 'max_iter': 1000}",4.770705,0.295176,6.047095,7.847101,0.458356,0.460365,0.098523,9.498697,0.525808
8,"{'hidden_layer_sizes': 32, 'max_iter': 1500}",4.628095,0.286991,5.891623,7.995462,0.495639,0.466947,0.095751,9.698625,0.572901
9,"{'hidden_layer_sizes': (8, 8), 'max_iter': 500}",5.020207,0.306725,6.329784,7.705481,0.456041,0.455848,0.099730,9.381973,0.484795


#### Random Forest

In [9]:
# MAE, MAPE
from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_absolute_error

# Normalizamos los datos
scaler = preprocessing.StandardScaler().fit(X)
X_norm = scaler.transform(X)

random.seed(19680801)

iterations = 10
gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
index = gss.split(X, Y, G)

model = RandomForestRegressor()

parameters = {'n_estimators': [5,10,20,50,100],
             'max_features': [5,10,16]}

# greater_is_better=True by default
mae = make_scorer(mean_absolute_error)
mape = make_scorer(MAPE)

scores = {'mae':mae,'mape':mape}

# Run the grid search
grid_obj = GridSearchCV(model, parameters, scoring=scores, cv=index, return_train_score=True, refit=False)
grid_obj = grid_obj.fit(X_norm, Y)


In [10]:
outcomes = pd.DataFrame(grid_obj.cv_results_)
outcomes = outcomes[['params', 'mean_train_mae', 'mean_train_mape',
            'mean_test_mae', 'std_test_mae', 'mean_test_mape', 'std_test_mape']]
outcomes

,params,mean_train_mae,mean_train_mape,mean_test_mae,std_test_mae,mean_test_mape,std_test_mape
0,"{'max_features': 5, 'n_estimators': 5}",2.154013,0.128760,7.736161,0.342465,0.458462,0.083408
1,"{'max_features': 5, 'n_estimators': 10}",2.004675,0.121814,7.567269,0.367710,0.450929,0.086070
2,"{'max_features': 5, 'n_estimators': 20}",1.893591,0.116413,7.403156,0.367118,0.443255,0.080878
3,"{'max_features': 5, 'n_estimators': 50}",1.820054,0.112088,7.324262,0.330162,0.438989,0.084031
4,"{'max_features': 5, 'n_estimators': 100}",1.802447,0.111402,7.320253,0.334875,0.439408,0.084469
5,"{'max_features': 10, 'n_estimators': 5}",2.075158,0.124841,7.809514,0.303777,0.465035,0.090379
6,"{'max_features': 10, 'n_estimators': 10}",1.947880,0.118376,7.631766,0.313510,0.455357,0.086818
7,"{'max_features': 10, 'n_estimators': 20}",1.840819,0.112535,7.465276,0.329436,0.446287,0.085697
8,"{'max_features': 10, 'n_estimators': 50}",1.783124,0.109724,7.410669,0.360195,0.444222,0.087167
9,"{'max_features': 10, 'n_estimators': 100}",1.756294,0.108379,7.413100,0.351831,0.444205,0.086081


#### Máquinas de Vectores de Soporte

In [ ]:
# MAE, MAPE, RMSE

from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error

# Normalizamos los datos
scaler = preprocessing.StandardScaler().fit(X)
X_norm = scaler.transform(X)

random.seed(19680801)

iterations = 10
gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
index = gss.split(X, Y, G)

model = SVR()

parameters = {'kernel': ['linear', 'rbf'],
              'C': [0.01, 1, 10],
              'gamma': [0.01, 0.1, 1]}

#print(model.get_params().keys())

# métricas de error
# greater_is_better=True by default
mae = make_scorer(mean_absolute_error)
mape = make_scorer(MAPE)
rmse = make_scorer(mean_squared_error, squared=False)

scores = {'mae':mae,'mape':mape,'rmse':rmse}

# Run the grid search
grid_obj = GridSearchCV(model, parameters, scoring=scores, cv=index, return_train_score=True, refit=False, verbose=12)
grid_obj = grid_obj.fit(X_norm, Y)

In [19]:
outcomes = pd.DataFrame(grid_obj.cv_results_)
outcomes = outcomes[['params', 'mean_train_mae', 'mean_train_mape', 'mean_train_rmse',
            'mean_test_mae', 'std_test_mae', 'mean_test_mape', 'std_test_mape', 'mean_test_rmse', 'std_test_rmse']]
outcomes

,params,mean_train_mae,mean_train_mape,mean_train_rmse,mean_test_mae,std_test_mae,mean_test_mape,std_test_mape,mean_test_rmse,std_test_rmse
0,"{'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}",6.378947,0.369702,7.796932,7.223411,0.583880,0.409434,0.068556,8.570458,0.774820
1,"{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}",6.867655,0.413149,8.105998,7.168634,0.739670,0.421469,0.084163,8.441721,0.782439
2,"{'C': 0.01, 'gamma': 0.1, 'kernel': 'linear'}",6.378947,0.369702,7.796932,7.223411,0.583880,0.409434,0.068556,8.570458,0.774820
3,"{'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}",6.755610,0.406680,7.993980,7.201606,0.682692,0.423462,0.081737,8.456004,0.745043
4,"{'C': 0.01, 'gamma': 1, 'kernel': 'linear'}",6.378947,0.369702,7.796932,7.223411,0.583880,0.409434,0.068556,8.570458,0.774820
5,"{'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}",6.835743,0.412721,8.067303,7.160230,0.718521,0.422686,0.086172,8.440372,0.755263
6,"{'C': 1, 'gamma': 0.01, 'kernel': 'linear'}",6.223547,0.361455,7.706775,7.487611,0.467114,0.420068,0.067237,9.123859,0.705711
7,"{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}",6.128618,0.357069,7.577974,7.327706,0.528497,0.414816,0.068232,8.718466,0.683757
8,"{'C': 1, 'gamma': 0.1, 'kernel': 'linear'}",6.223547,0.361455,7.706775,7.487611,0.467114,0.420068,0.067237,9.123859,0.705711
9,"{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}",5.264087,0.317361,6.749162,7.455751,0.452823,0.431873,0.080684,8.953434,0.535530
